# Visualising networks using nglview

In [1]:
import kugupu as kgp
import MDAnalysis as mda
import nglview as nv
import numpy as np

In [2]:
res = kgp.load_results('datafiles/results.hdf5')

In [3]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.guesser.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e

In [4]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [5]:
add_names(u)

## Visualising a network

Firstly an entire molecule network can be visualised using the `visualise.draw_networks` functions.

First we calculate the networks present in the first frame of results, here at a threshold of 1meV.

In [6]:
nets = kgp.networks.find_networks(u.atoms.fragments, res.H_frag[0], res.degeneracy, 0.001)

We then pass the first (and largest) network in the system to the `draw_network` function.

This function makes sure that all molecules are "whole", i.e. their bonds are not split over periodic boundaries, and then moves all molecules between periodic images to minimise the distance between their centers of mass (this is done using a minimum spanning tree).
The end result is the most compact representation.

Finally the network is drawn over the molecules as red lines between the centers of electronically coupled molecules.

In [7]:
v = kgp.visualise.draw_network(nets[0], show_molecules=True)
v.add_unitcell()
v

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/lib/util.py:708: RuntimeWarning: Constructed NamedStream from a NamedStream
  warnings.warn(
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/topology/PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/core/universe.py:743: UserWarning: Reader has no dt information, set to 1.0 ps
  dt=self.trajectory.ts.dt * step,


NGLWidget()

## Visualising a particular dimer

We might also be interested in visualising a particular dimer pair, perhaps because of their strong coupling.

In [8]:
print("The largest coupling is: {:.3f} eV".format(res.H_frag[0].max()))

The largest coupling is: 0.226 eV


Here we determine the position in the coupling matrix of the largest coupling value:

In [9]:
pos = np.argmax(res.H_frag[0])

x, y = pos // res.H_frag.shape[1], pos % res.H_frag.shape[2]

In [10]:
res.H_frag[0, x, y]

np.float64(0.22601396484141995)

We can then draw the fragments by passing these fragments (indexed by the variables `x` and `y`) to the `visualise.draw_fragments` functions.

In [11]:
v = kgp.visualise.draw_fragments(u.atoms.fragments[x], u.atoms.fragments[y])
v.add_unitcell()
v

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/lib/util.py:708: RuntimeWarning: Constructed NamedStream from a NamedStream
  warnings.warn(
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/topology/PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.13/site-packages/MDAnalysis/core/universe.py:743: UserWarning: Reader has no dt information, set to 1.0 ps
  dt=self.trajectory.ts.dt * step,


NGLWidget()